In [1]:
#reading pdf and saving it into txt

In [9]:

import fitz
pdf_path=  'pdf/_OceanofPDF.com_Mahabharata_Unravelled_-_Ami_Ganatra.pdf'

pdf_doc = fitz.open(pdf_path)

with open('output.txt','w',encoding = 'utf-8') as file:
    for page_number in range(len(pdf_doc)):
        page = pdf_doc[page_number]
        text = page.get_text()
        file.write(f"Page {page_number+1}:\n{text}\n")
pdf_doc.close()

In [10]:
#llm buuilding

In [11]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [15]:
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [16]:
model = "gpt-4o-mini"
db_name = "vector_db"

In [17]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [19]:
txt_path = 'output.txt'
with open(txt_path,'r',encoding  =' utf-8') as file:
    content = file.read()

In [23]:
#loading txt using langchain loader
txt_path = 'output.txt'
loader = TextLoader(txt_path,encoding ="utf-8")
content = loader.load()

In [24]:
text_splitter = CharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
chunks = text_splitter.split_documents(content)

Created a chunk of size 1503, which is longer than the specified 1000
Created a chunk of size 2005, which is longer than the specified 1000
Created a chunk of size 1800, which is longer than the specified 1000
Created a chunk of size 1943, which is longer than the specified 1000
Created a chunk of size 1978, which is longer than the specified 1000
Created a chunk of size 1382, which is longer than the specified 1000
Created a chunk of size 1426, which is longer than the specified 1000
Created a chunk of size 1835, which is longer than the specified 1000
Created a chunk of size 1790, which is longer than the specified 1000
Created a chunk of size 1630, which is longer than the specified 1000
Created a chunk of size 2075, which is longer than the specified 1000
Created a chunk of size 1892, which is longer than the specified 1000
Created a chunk of size 1744, which is longer than the specified 1000
Created a chunk of size 2067, which is longer than the specified 1000
Created a chunk of s

In [25]:
len(chunks)

317

In [27]:
embeddings = OpenAIEmbeddings()

In [28]:
if os.path.exists(db_name):
    Chroma(persist_directory = db_name,embedding_function = embeddings).delete_collection()

In [31]:
vectorstore = Chroma.from_documents(documents = chunks,embedding = embeddings,persist_directory = db_name)
print(f"vectorstore created with {vectorstore._collection.count()} documents")

vectorstore created with 317 documents


In [32]:
#llm_model bulding

In [33]:
llm = ChatOpenAI(temperature = 0.5,model_name= model)
memory = ConversationBufferMemory(memory_key = 'chat_history',return_message = True)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(llm = llm,retriever = retriever,memory = memory)

C:\Users\prana\AppData\Local\Temp\ipykernel_15244\2790693016.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = 'chat_history',return_message = True)


In [35]:
query = 'Who were the parents of pandavas'
result=  conversation_chain.invoke({'question':query})
print(result['answer'])

The parents of the Pandavas were King Pāndu and his wives Kunti and Mādri. Kunti was the daughter of Shurasena of the Yādava clan, and Mādri was the sister of Shalya, the king of Madra desh.


In [36]:
#putting it together into chat window

In [37]:
memory = ConversationBufferMemory(memory_key = 'chat_history',return_messages = True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm = llm,retriever = retriever,memory = memory)
def chat(message,history):
    result = conversation_chain.invoke({'question':message})
    return result['answer']

In [38]:
view = gr.ChatInterface(chat,type = 'messages').launch(inbrowser = True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
